In [1]:
!python -V

Python 3.9.18


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
from sklearn.metrics import root_mean_squared_error

In [9]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='file:///e:/algorithmic_trading/python/git_repo/ZoomCamp/MLOps/notes_class/week2/mlruns/1', creation_time=1716497349409, experiment_id='1', last_update_time=1716497349409, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [10]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
        df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
        df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
        
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [11]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

In [12]:
len(df_train), len(df_val)

(3009173, 2855951)

In [13]:
categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [15]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

e:\algorithmic_trading\miniconda3\envs\zoomcamp\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.811816520976144

In [17]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

# Hot to use mlflow

In [18]:
with mlflow.start_run():

    mlflow.set_tag("developer", "jhon")

    mlflow.log_param("train-data-path", "./data/yellow_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path", "./data/yellow_tripdata_2023-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

e:\algorithmic_trading\miniconda3\envs\zoomcamp\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [19]:
import xgboost as xgb

In [20]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [21]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():

        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, 'validation')],
            early_stopping_rounds=2
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)

        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [24]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

[11:59:56] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.38641                         
[1]	validation-rmse:15.61422                         
[2]	validation-rmse:14.89501                         
[3]	validation-rmse:14.22821                         
[4]	validation-rmse:13.58612                         
[5]	validation-rmse:13.01378                         
[6]	validation-rmse:12.48361                         
[7]	validation-rmse:11.94665                         
[8]	validation-rmse:11.45146                         
[9]	validation-rmse:10.97113                         
[12:00:21] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror

e:\algorithmic_trading\miniconda3\envs\zoomcamp\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:13.55023                                                   
[1]	validation-rmse:11.04430                                                   
[2]	validation-rmse:9.37774                                                    
[3]	validation-rmse:7.87511                                                    
[4]	validation-rmse:6.89748                                                    
[5]	validation-rmse:6.41556                                                    
[6]	validation-rmse:6.13628                                                    
[7]	validation-rmse:5.79697                                                    
[8]	validation-rmse:5.69075                                                    
[9]	validation-rmse:5.52174                                                    
[12:00:55] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated

e:\algorithmic_trading\miniconda3\envs\zoomcamp\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:16.41593                                                  
[1]	validation-rmse:15.67016                                                  
[2]	validation-rmse:14.97599                                                  
[3]	validation-rmse:14.32671                                                  
[4]	validation-rmse:13.72551                                                  
[5]	validation-rmse:13.16739                                                  
[6]	validation-rmse:12.65047                                                  
[7]	validation-rmse:12.11003                                                  
[8]	validation-rmse:11.63041                                                  
[9]	validation-rmse:11.15880                                                  
[12:01:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor 

e:\algorithmic_trading\miniconda3\envs\zoomcamp\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:15.08004                                                  
[1]	validation-rmse:13.32574                                                  
[2]	validation-rmse:11.89875                                                  
[3]	validation-rmse:10.58719                                                  
[4]	validation-rmse:9.66122                                                   
[5]	validation-rmse:8.71894                                                   
[6]	validation-rmse:7.96558                                                   
[7]	validation-rmse:7.37139                                                   
[8]	validation-rmse:6.98894                                                   
[9]	validation-rmse:6.70497                                                   
[12:01:51] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor 

e:\algorithmic_trading\miniconda3\envs\zoomcamp\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:8.42815                                                   
[1]	validation-rmse:7.17743                                                   
[2]	validation-rmse:6.23305                                                   
[3]	validation-rmse:6.13853                                                   
[4]	validation-rmse:6.00437                                                   
[5]	validation-rmse:5.88096                                                   
[6]	validation-rmse:5.79231                                                   
[7]	validation-rmse:5.76037                                                   
[8]	validation-rmse:5.63299                                                   
[9]	validation-rmse:5.61855                                                   
100%|██████████| 5/5 [02:11<00:00, 26.39s/trial, best loss: 5.521743673499381]


e:\algorithmic_trading\miniconda3\envs\zoomcamp\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [25]:
mlflow.xgboost.autolog()

with mlflow.start_run():

    best_params = {
        'learning_rate': 0.7551525030604964,
        'max_depth': 41,
        'min_child_weight': 11.292666844643884,
        'objective': 'reg:linear',
        'reg_alpha': 0.04230842744726431,
        'reg_lambda': 0.2462273673862625,
        'seed': 42
    }

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=10,
        evals=[(valid, 'validation')],
        early_stopping_rounds=2
    )


[12:21:38] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:8.42815
[1]	validation-rmse:7.17743
[2]	validation-rmse:6.23305
[3]	validation-rmse:6.13853
[4]	validation-rmse:6.00437
[5]	validation-rmse:5.88096
[6]	validation-rmse:5.79231
[7]	validation-rmse:5.76037
[8]	validation-rmse:5.63299
[9]	validation-rmse:5.61855


2024/06/07 12:21:55 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/06/07 12:22:01 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "e:\algorithmic_trading\miniconda3\envs\zoomcamp\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [27]:
mlflow.xgboost.autolog(disable=True)

In [29]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.7551525030604964,
        'max_depth': 41,
        'min_child_weight': 11.292666844643884,
        'objective': 'reg:linear',
        'reg_alpha': 0.04230842744726431,
        'reg_lambda': 0.2462273673862625,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=10,
        evals=[(valid, 'validation')],
        early_stopping_rounds=2
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[15:55:34] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:8.42815
[1]	validation-rmse:7.17743
[2]	validation-rmse:6.23305
[3]	validation-rmse:6.13853
[4]	validation-rmse:6.00437
[5]	validation-rmse:5.88096
[6]	validation-rmse:5.79231
[7]	validation-rmse:5.76037
[8]	validation-rmse:5.63299
[9]	validation-rmse:5.61855


e:\algorithmic_trading\miniconda3\envs\zoomcamp\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
